In [9]:
#!/usr/bin/env python3
"""
Raman Spectroscopy Analysis for Choline Phosphate.

This module provides temperature-dependent conformational analysis with interactive
peak picking for Raman spectroscopy data. It supports automatic conformer assignment
(gauche, anti, dihydrogen phosphate), Voigt profile fitting, and comprehensive
visualization of results.

Features:
    - Interactive peak selection via GUI
    - Two-round fitting with residual-based outlier detection
    - Automatic baseline correction (arPLS)
    - Voigt profile fitting (Gaussian + Lorentzian)
    - Temperature-resolved spectral analysis
    - Conformer ratio calculation (excluding phosphate peaks)

Author: Janis Hessling
Date: 2025
Version: 1.0

GitHub: https://github.com/janishessling/choline-conformer-raman-analysis
License: MIT
"""

# Standard library imports
import gc
import os
import re
import time
from typing import Dict, List, Optional, Tuple

# Third-party imports
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.widgets import Button
import numpy as np
import pandas as pd
from lmfit import Parameters, minimize as lmfit_minimize
from scipy.integrate import simpson
import tkinter as tk
from tkinter import filedialog

# Module-level constants
# Peak classification thresholds (cm⁻¹)
GAUCHE_THRESHOLD = 740.0
PHOSPHATE_THRESHOLD = 790.0
PEAK_REGION_MIN = 680
PEAK_REGION_MAX = 810

# Peak fitting parameters
PEAK_WIDTH_MIN = 1  # cm⁻¹
PEAK_WIDTH_MAX = 20  # cm⁻¹
PEAK_POSITION_TOLERANCE = 5  # cm⁻¹
INITIAL_PEAK_WIDTH = 8  # cm⁻¹
AMPLITUDE_MAX_FACTOR = 3
BASELINE_MAX_FACTOR = 0.1

# Outlier detection
OUTLIER_THRESHOLD_SIGMA = 5.0  # 3σ = standard, 2.5 = aggressive, 5.0 = conservative

# Plot styling constants for consistent visualization
PLOT_LINEWIDTH_THICK = 2.5
PLOT_LINEWIDTH_NORMAL = 2.0
PLOT_LINEWIDTH_THIN = 1.5
PLOT_ALPHA_HIGH = 0.8
PLOT_ALPHA_MEDIUM = 0.7
PLOT_ALPHA_LOW = 0.3
PLOT_FONTSIZE_TITLE = 14
PLOT_FONTSIZE_LABEL = 13
PLOT_FONTSIZE_LEGEND = 9
PLOT_FONTSIZE_SMALL = 8
PLOT_MARKER_SIZE_LARGE = 150
PLOT_MARKER_SIZE_MEDIUM = 80

# Output settings
OUTPUT_DPI = 300

# GUI constants for consistent interface
GUI_WINDOW_WIDTH = 520
GUI_WINDOW_HEIGHT = 500


# =============================================================================
# GUI FUNCTIONS
# =============================================================================

def select_data_folder() -> str:
    """
    Open file dialog for folder selection with validation.
    
    Returns:
        str: Path to selected folder containing Raman data files.
        
    Raises:
        SystemExit: If no folder is selected or folder is invalid.
    """
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    
    print("=" * 80)
    print("FOLDER SELECTION")
    print("=" * 80)
    print("Please select the folder containing your Raman spectroscopy data...")
    
    folder_path = filedialog.askdirectory(
        title="Select Folder with Raman Data",
        mustexist=True
    )
    
    root.destroy()
    
    if not folder_path:
        print("\n✗ No folder selected. Exiting...")
        raise SystemExit(0)
    
    # Validate folder exists and is accessible
    if not os.path.exists(folder_path):
        print(f"\n✗ Error: Selected folder does not exist: {folder_path}")
        raise SystemExit(1)
    
    if not os.path.isdir(folder_path):
        print(f"\n✗ Error: Selected path is not a folder: {folder_path}")
        raise SystemExit(1)
    
    # Check if folder contains .txt files
    try:
        txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
        if not txt_files:
            print(f"\n⚠️  Warning: No .txt files found in selected folder")
            print("    Make sure your Raman data files have .txt extension")
        else:
            print(f"  Found {len(txt_files)} .txt file(s)")
    except PermissionError:
        print(f"\n✗ Error: No permission to access folder: {folder_path}")
        raise SystemExit(1)
    
    print(f"\n✓ Selected folder: {folder_path}")
    print("=" * 80)
    
    return folder_path




def _create_temperature_window() -> tk.Tk:
    """
    Create and configure the main temperature range configuration window.
    
    Returns:
        tk.Tk: Configured root window with centered position
    """
    # Force cleanup before creating new window
    plt.close('all')
    gc.collect()
    time.sleep(0.1)
    
    root = tk.Tk()
    root.title("Temperature Range Configuration")
    root.geometry(f"{GUI_WINDOW_WIDTH}x{GUI_WINDOW_HEIGHT}")
    root.resizable(True, True)
    root.attributes('-topmost', True)
    
    # Center window on screen
    root.update_idletasks()
    x = (root.winfo_screenwidth() // 2) - (GUI_WINDOW_WIDTH // 2)
    y = (root.winfo_screenheight() // 2) - (GUI_WINDOW_HEIGHT // 2)
    root.geometry(f"{GUI_WINDOW_WIDTH}x{GUI_WINDOW_HEIGHT}+{x}+{y}")
    
    return root


def _create_temperature_widgets(
    root: tk.Tk,
    filter_min: tk.DoubleVar,
    filter_max: tk.DoubleVar,
    color_min: tk.DoubleVar,
    color_max: tk.DoubleVar
) -> Tuple[List[tk.Entry], List[tk.Label]]:
    """
    Create all input widgets for temperature configuration.
    
    Args:
        root: Parent window
        filter_min: Variable for minimum filter temperature
        filter_max: Variable for maximum filter temperature
        color_min: Variable for minimum color scale temperature
        color_max: Variable for maximum color scale temperature
        
    Returns:
        Tuple of (entry_widgets, label_widgets) for state management
    """
    # Create title
    title_label = tk.Label(
        root, 
        text="Temperature Range Configuration",
        font=('Arial', 13, 'bold')
    )
    title_label.pack(pady=15)
    
    # Main input frame
    main_input_frame = tk.Frame(root)
    main_input_frame.pack(pady=10, padx=20)
    
    # Data filtering section
    filter_title = tk.Label(
        main_input_frame,
        text="1. Temperature range to analyze:",
        font=('Arial', 10, 'bold')
    )
    filter_title.grid(row=0, column=0, columnspan=3, sticky='w', pady=(5, 5))
    
    filter_label1 = tk.Label(
        main_input_frame,
        text="  Min temp (°C):",
        font=('Arial', 9)
    )
    filter_label1.grid(row=1, column=0, sticky='e', padx=5, pady=3)
    
    filter_min_entry = tk.Entry(
        main_input_frame,
        textvariable=filter_min,
        width=10,
        font=('Arial', 9)
    )
    filter_min_entry.grid(row=1, column=1, padx=5, pady=3)
    
    tk.Label(
        main_input_frame,
        text="(data outside this range excluded)",
        font=('Arial', 8),
        fg='gray'
    ).grid(row=1, column=2, sticky='w', padx=5)
    
    filter_label2 = tk.Label(
        main_input_frame,
        text="  Max temp (°C):",
        font=('Arial', 9)
    )
    filter_label2.grid(row=2, column=0, sticky='e', padx=5, pady=3)
    
    filter_max_entry = tk.Entry(
        main_input_frame,
        textvariable=filter_max,
        width=10,
        font=('Arial', 9)
    )
    filter_max_entry.grid(row=2, column=1, padx=5, pady=3)
    
    # Separator
    tk.Frame(
        main_input_frame,
        height=2,
        bg='lightgray'
    ).grid(row=3, column=0, columnspan=3, sticky='ew', pady=10)
    
    # Color scale section
    color_title = tk.Label(
        main_input_frame,
        text="2. Temperature range for color scale:",
        font=('Arial', 10, 'bold')
    )
    color_title.grid(row=4, column=0, columnspan=3, sticky='w', pady=(5, 5))
    
    color_label1 = tk.Label(
        main_input_frame,
        text="  Min temp (°C):",
        font=('Arial', 9)
    )
    color_label1.grid(row=5, column=0, sticky='e', padx=5, pady=3)
    
    color_min_entry = tk.Entry(
        main_input_frame,
        textvariable=color_min,
        width=10,
        font=('Arial', 9)
    )
    color_min_entry.grid(row=5, column=1, padx=5, pady=3)
    
    tk.Label(
        main_input_frame,
        text="(blue color)",
        font=('Arial', 8),
        fg='blue'
    ).grid(row=5, column=2, sticky='w', padx=5)
    
    color_label2 = tk.Label(
        main_input_frame,
        text="  Max temp (°C):",
        font=('Arial', 9)
    )
    color_label2.grid(row=6, column=0, sticky='e', padx=5, pady=3)
    
    color_max_entry = tk.Entry(
        main_input_frame,
        textvariable=color_max,
        width=10,
        font=('Arial', 9)
    )
    color_max_entry.grid(row=6, column=1, padx=5, pady=3)
    
    tk.Label(
        main_input_frame,
        text="(red color)",
        font=('Arial', 8),
        fg='red'
    ).grid(row=6, column=2, sticky='w', padx=5)
    
    # Info label
    info_label = tk.Label(
        root,
        text="Example: Analyze 15-140°C but use -40 to 140°C color scale\n"
             "for comparison with other datasets",
        font=('Arial', 8),
        fg='gray',
        justify='center'
    )
    info_label.pack(pady=10)
    
    # Store all widgets that need state changes
    entry_widgets = [filter_min_entry, filter_max_entry, 
                     color_min_entry, color_max_entry]
    label_widgets = [filter_label1, filter_label2, color_label1, color_label2,
                     filter_title, color_title]
    
    return entry_widgets, label_widgets


def _setup_radio_buttons_and_callbacks(
    root: tk.Tk,
    use_custom: tk.BooleanVar,
    entry_widgets: List[tk.Entry],
    label_widgets: List[tk.Label]
) -> tk.Frame:
    """
    Setup radio buttons and their callbacks for mode selection.
    
    Args:
        root: Parent window
        use_custom: Variable for custom/auto mode selection
        entry_widgets: List of entry widgets to enable/disable
        label_widgets: List of label widgets to enable/disable
        
    Returns:
        tk.Frame: Radio button frame
    """
    radio_frame = tk.Frame(root)
    radio_frame.pack(pady=10)
    
    def update_entry_states():
        """Enable or disable input fields based on mode selection."""
        if use_custom.get():
            # Manual mode - enable everything
            for entry in entry_widgets:
                entry.config(state='normal', fg='black', bg='white')
            for label in label_widgets:
                label.config(fg='black')
        else:
            # Automatic mode - disable everything
            for entry in entry_widgets:
                entry.config(state='disabled', fg='lightgrey', bg='#f0f0f0')
            for label in label_widgets:
                label.config(fg='lightgrey')
        
        # Force immediate update
        root.update()
    
    # Radio buttons
    auto_radio = tk.Radiobutton(
        radio_frame,
        text="Automatic (use all data, auto color scale)",
        variable=use_custom,
        value=False,
        font=('Arial', 10),
        command=update_entry_states
    )
    auto_radio.pack(anchor='w')
    
    custom_radio = tk.Radiobutton(
        radio_frame,
        text="Manual (set custom ranges):",
        variable=use_custom,
        value=True,
        font=('Arial', 10),
        command=update_entry_states
    )
    custom_radio.pack(anchor='w', pady=5)
    
    # Set initial state (disabled)
    update_entry_states()
    
    return radio_frame


def _cleanup_tk_resources(*resources) -> None:
    """
    Safely cleanup Tkinter resources with proper error handling.
    
    Args:
        *resources: Variable number of Tkinter resources to clean up
    """
    for resource in resources:
        try:
            if hasattr(resource, 'quit'):
                resource.quit()
        except Exception:
            pass
        
        try:
            if hasattr(resource, 'destroy'):
                resource.destroy()
        except Exception:
            pass
        
        try:
            del resource
        except Exception:
            pass
    
    gc.collect()
    time.sleep(0.2)


def select_temperature_range() -> Tuple[Optional[float], Optional[float], 
                                       Optional[float], Optional[float]]:
    """
    Open dialog for temperature range configuration.
    
    This function has been refactored into smaller helper functions for better
    maintainability. The GUI logic is now split into:
    - _create_temperature_window(): Window creation and positioning
    - _create_temperature_widgets(): Widget creation
    - _setup_radio_buttons_and_callbacks(): Event handling
    - _cleanup_tk_resources(): Resource cleanup
    
    Returns:
        Tuple containing:
            - filter_temp_min: Minimum temperature for data filtering (None for auto)
            - filter_temp_max: Maximum temperature for data filtering (None for auto)
            - color_temp_min: Minimum temperature for color scale (None for auto)
            - color_temp_max: Maximum temperature for color scale (None for auto)
    """
    print("\n" + "=" * 80)
    print("TEMPERATURE RANGE CONFIGURATION")
    print("=" * 80)
    print("Configure temperature ranges for analysis and color coding")
    
    # Create main window
    root = _create_temperature_window()
    
    # Initialize variables
    use_custom = tk.BooleanVar(master=root, value=False)
    filter_min = tk.DoubleVar(master=root, value=-40.0)
    filter_max = tk.DoubleVar(master=root, value=140.0)
    color_min = tk.DoubleVar(master=root, value=-40.0)
    color_max = tk.DoubleVar(master=root, value=140.0)
    result = {'confirmed': False}
    
    # Create widgets
    entry_widgets, label_widgets = _create_temperature_widgets(
        root, filter_min, filter_max, color_min, color_max
    )
    
    # Setup radio buttons and callbacks
    radio_frame = _setup_radio_buttons_and_callbacks(
        root, use_custom, entry_widgets, label_widgets
    )
    
    def confirm():
        """Store user configuration and close dialog."""
        result['confirmed'] = True
        result['use_custom'] = use_custom.get()
        result['filter_min'] = filter_min.get()
        result['filter_max'] = filter_max.get()
        result['color_min'] = color_min.get()
        result['color_max'] = color_max.get()
        root.quit()
    
    def on_closing():
        """Handle window close event."""
        result['confirmed'] = False
        root.quit()
    
    # Bind window close event
    root.protocol("WM_DELETE_WINDOW", on_closing)
    
    # OK button
    ok_button = tk.Button(
        root,
        text="OK - Continue Analysis",
        command=confirm,
        font=('Arial', 12, 'bold'),
        bg='#90EE90',
        width=25,
        height=2,
        cursor='hand2'
    )
    ok_button.pack(pady=20)
    
    # Make sure window is fully rendered before starting mainloop
    root.update()
    
    # Run dialog
    try:
        root.mainloop()
    except Exception as e:
        print(f"Dialog error: {e}")
    
    # Complete cleanup
    _cleanup_tk_resources(root, use_custom, filter_min, filter_max, 
                         color_min, color_max)
    
    # Return results
    if not result['confirmed']:
        print("\n✗ Dialog closed without confirmation. Using automatic mode.")
        return None, None, None, None
    
    if result['use_custom']:
        print(f"\n✓ Manual mode selected:")
        print(f"  Data range: {result['filter_min']:.0f} to "
              f"{result['filter_max']:.0f} °C")
        print(f"  Color scale: {result['color_min']:.0f} to "
              f"{result['color_max']:.0f}°C")
        return (result['filter_min'], result['filter_max'],
                result['color_min'], result['color_max'])
    else:
        print("\n✓ Automatic mode selected (use all data, auto color scale)")
        return None, None, None, None


# =============================================================================
# BASELINE CORRECTION
# =============================================================================

def baseline_correct_with_linear_shift(
    x: np.ndarray,
    y: np.ndarray,
    method: str = 'arPLS',
    lam: float = 1e6,
    ratio: float = 0.01,
    percentile_threshold: int = 25,
    target_min: float = 0.0,
    use_linear: bool = True
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Perform baseline correction with intelligent linear shift.
    
    Uses rampy library for initial baseline estimation, then applies
    a linear shift correction to handle negative values or baseline
    overshooting issues.
    
    Args:
        x: Wavenumber array
        y: Intensity array
        method: Baseline correction method ('arPLS', 'drPLS', 'als')
        lam: Smoothness parameter for baseline algorithm
        ratio: Asymmetry ratio parameter
        percentile_threshold: Percentile for problematic region detection
        target_min: Target minimum value after correction
        use_linear: Whether to apply linear shift correction
        
    Returns:
        Tuple of (corrected intensity array, adjusted baseline array)
    """
    import rampy as rp
    from scipy import stats as scipy_stats
    
    roi = np.array([[x.min(), x.max()]])
    
    # Apply baseline correction
    if method in ['arPLS', 'drPLS']:
        y_corr, baseline = rp.baseline(x, y, roi, method, lam=lam, ratio=ratio)
    elif method == 'als':
        y_corr, baseline = rp.baseline(x, y, roi, method, lam=lam, p=ratio)
    else:
        y_corr, baseline = rp.baseline(x, y, roi, method)
    
    y_corr = np.squeeze(y_corr)
    baseline = np.squeeze(baseline)
    
    # Detect problematic regions
    baseline_exceeds = baseline > y
    negative_mask = y_corr < 0
    problem_mask = negative_mask | baseline_exceeds
    
    # Apply linear shift if needed
    if np.sum(problem_mask) > 0 and use_linear:
        problem_indices = np.where(problem_mask)[0]
        x_problem = x[problem_indices]
        y_problem = y_corr[problem_indices]
        
        if len(y_problem) > 0:
            threshold_value = np.percentile(y_problem, percentile_threshold)
            fit_mask = y_problem <= threshold_value
            
            if np.sum(fit_mask) >= 2:
                x_fit = x_problem[fit_mask]
                y_fit = y_problem[fit_mask]
                target_shift = target_min - y_fit
                slope, intercept, _, _, _ = scipy_stats.linregress(
                    x_fit, target_shift
                )
                m = slope
                b = intercept
            else:
                m = 0
                b = target_min - np.min(y_problem)
        else:
            m = 0
            b = 0
    else:
        m = 0
        b = 0
    
    linear_shift = m * x + b
    baseline_adjusted = baseline - linear_shift
    y_corrected = y - baseline_adjusted
    
    return y_corrected, baseline_adjusted


# =============================================================================
# OUTLIER DETECTION (RESIDUAL-BASED AFTER FIRST FIT)
# =============================================================================

def detect_outliers_from_residuals(
    y_measured: np.ndarray,
    y_fitted: np.ndarray,
    threshold_sigma: float = None
) -> np.ndarray:
    """
    Detect outliers based on fit residuals using robust statistics (MAD).
    
    This method identifies cosmic rays and detector artifacts by comparing
    measured intensities to the fitted model. Points with large residuals
    are flagged as outliers.
    
    Args:
        y_measured: Measured intensity array
        y_fitted: Fitted intensity from first round
        threshold_sigma: Number of sigma for outlier threshold (default: uses OUTLIER_THRESHOLD_SIGMA constant)
        
    Returns:
        Boolean array (True = outlier detected)
    """
    # Use module constant if not specified
    if threshold_sigma is None:
        threshold_sigma = OUTLIER_THRESHOLD_SIGMA
    
    # Calculate residuals
    residuals = y_measured - y_fitted
    
    # Use Median Absolute Deviation (MAD) - robust to outliers
    median_residual = np.median(residuals)
    mad = np.median(np.abs(residuals - median_residual))
    
    # Convert MAD to equivalent standard deviation
    sigma_robust = 1.4826 * mad
    
    # Avoid division by zero
    if sigma_robust < 1e-10:
        return np.zeros(len(y_measured), dtype=bool)
    
    # Flag points with residuals > threshold_sigma * sigma
    outlier_mask = np.abs(residuals - median_residual) > (threshold_sigma * sigma_robust)
    
    return outlier_mask


def plot_outlier_detection(
    x: np.ndarray,
    y_measured: np.ndarray,
    y_fitted: np.ndarray,
    outlier_mask: np.ndarray,
    temperature: float,
    filename: str,
    output_folder: str
):
    """
    Create visualization of detected outliers from fit residuals.
    
    Args:
        x: Wavenumber array
        y_measured: Measured intensity array
        y_fitted: Fitted intensity from round 1
        outlier_mask: Boolean array indicating outlier positions
        temperature: Measurement temperature
        filename: Original filename
        output_folder: Folder for saving plots
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Top panel: Spectrum with fit and outliers
    ax1.plot(x, y_measured, 'b-', linewidth=1.5, label='Measured Spectrum', alpha=0.7)
    ax1.plot(x, y_fitted, 'r--', linewidth=2, label='Round 1 Fit', alpha=0.8)
    
    # Mark outliers if any detected
    if np.any(outlier_mask):
        ax1.scatter(
            x[outlier_mask], y_measured[outlier_mask],
            c='red', s=150, marker='x', linewidths=3,
            label=f'Detected Outliers (n={np.sum(outlier_mask)})',
            zorder=5
        )
    
    ax1.set_xlabel('Wavenumber / cm⁻¹', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Raman Intensity', fontsize=12, fontweight='bold')
    ax1.set_title(
        f'Outlier Detection (Residual-Based): {temperature:.0f}°C\n{filename}',
        fontsize=11,
        fontweight='bold'
    )
    ax1.legend(fontsize=10, loc='best')
    ax1.grid(True, alpha=0.3)
    
    # Bottom panel: Residuals
    residuals = y_measured - y_fitted
    ax2.plot(x, residuals, 'g-', linewidth=1.5, label='Residuals', alpha=0.7)
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)
    
    # Show threshold lines (uses OUTLIER_THRESHOLD_SIGMA constant)
    if np.any(~outlier_mask):  # If we have non-outlier points
        median_residual = np.median(residuals[~outlier_mask])
        mad = np.median(np.abs(residuals[~outlier_mask] - median_residual))
        sigma_robust = 1.4826 * mad
        
        threshold = OUTLIER_THRESHOLD_SIGMA * sigma_robust
        ax2.axhline(y=median_residual + threshold, color='red', 
                   linestyle=':', linewidth=2, alpha=0.5,
                   label=f'±{OUTLIER_THRESHOLD_SIGMA}σ threshold')
        ax2.axhline(y=median_residual - threshold, color='red', 
                   linestyle=':', linewidth=2, alpha=0.5)
    
    # Mark outlier residuals
    if np.any(outlier_mask):
        ax2.scatter(
            x[outlier_mask], residuals[outlier_mask],
            c='red', s=150, marker='x', linewidths=3,
            label='Outlier Residuals',
            zorder=5
        )
    
    ax2.set_xlabel('Wavenumber / cm⁻¹', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Residual', fontsize=12, fontweight='bold')
    ax2.set_title('Fit Residuals', fontsize=11, fontweight='bold')
    ax2.legend(fontsize=10, loc='best')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Create outlier detection subfolder if it doesn't exist
    outlier_folder = os.path.join(output_folder, 'Outlier_Detection')
    os.makedirs(outlier_folder, exist_ok=True)
    
    # Save plot
    output_filename = f'Outliers_{temperature:.0f}C.png'
    plt.savefig(
        os.path.join(outlier_folder, output_filename),
        dpi=150,
        bbox_inches='tight'
    )
    plt.close()


# =============================================================================
# INTERACTIVE PEAK PICKING
# =============================================================================

class InteractivePeakPicker:
    """
    Interactive peak picking tool for Raman spectra.
    
    Provides GUI for manual peak selection with visual feedback showing
    gauche (<740 cm⁻¹), anti (740-790 cm⁻¹), and phosphate (≥790 cm⁻¹) peaks.
    Outliers are visually marked and excluded from selection.
    
    Attributes:
        x: Wavenumber array
        y: Intensity array
        y_original: Original intensity before interpolation
        outlier_mask: Boolean array indicating outlier positions
        temperature: Measurement temperature
        filename: Source filename
        peaks: List of selected peak positions as (x, y) tuples
        done: Flag indicating completion
        skip: Flag indicating spectrum should be skipped
    """
    
    def __init__(
        self,
        x: np.ndarray,
        y: np.ndarray,
        temperature: float,
        filename: str
    ):
        """
        Initialize peak picker interface.
        
        Args:
            x: Wavenumber array
            y: Intensity array (cleaned)
            y_original: Original intensity array (before interpolation)
            outlier_mask: Boolean array indicating outlier positions
            temperature: Measurement temperature in °C
            filename: Source filename for display
        """
        self.x = x
        self.y = y
        self.temperature = temperature
        self.filename = filename
        self.peaks = []
        self.done = False
        self.skip = False
        
        self._setup_figure()
        self._setup_buttons()
        self._connect_events()
    
    def _setup_figure(self):
        """Create and configure the matplotlib figure."""
        self.fig = plt.figure(figsize=(12, 7))
        self.fig.canvas.manager.set_window_title(
            f'Peak Picker - {self.temperature:.0f}°C'
        )
        
        self.ax = self.fig.add_axes([0.1, 0.2, 0.85, 0.7])
        
        # Plot spectrum
        self.line, = self.ax.plot(
            self.x, self.y, 'b-', linewidth=2, label='Spectrum'
        )
        
        
        self.ax.set_xlabel('Wavenumber / cm⁻¹', fontsize=12, fontweight='bold')
        self.ax.set_ylabel('Raman Intensity', fontsize=12, fontweight='bold')
        self.ax.set_title(
            f'Peak Picking: {self.temperature:.0f}°C - Click on peak maxima\n'
            f'{self.filename}',
            fontsize=11,
            fontweight='bold'
        )
        self.ax.grid(True, alpha=0.3)
        
        # Add conformer boundary markers
        self.ax.axvline(
            x=GAUCHE_THRESHOLD,
            color='red',
            linestyle='--',
            linewidth=2,
            alpha=0.7,
            label=f'Gauche/Anti boundary ({GAUCHE_THRESHOLD:.0f} cm⁻¹)'
        )
        self.ax.axvline(
            x=PHOSPHATE_THRESHOLD,
            color='orange',
            linestyle='--',
            linewidth=2,
            alpha=0.7,
            label=f'Anti/H₂PO₄⁻ boundary ({PHOSPHATE_THRESHOLD:.0f} cm⁻¹)'
        )
        
        # Initialize peak markers
        self.peak_scatter = self.ax.scatter(
            [], [],
            c='green',
            s=150,
            marker='o',
            edgecolors='black',
            linewidths=2,
            label='Selected peaks',
            zorder=6
        )
        self.peak_labels = []
        
        # Add instruction text
        self.instruction_text = self.ax.text(
            0.02, 0.98,
            f'Click on peak maxima\nPeaks selected: 0\n'
            'Gauche (<740): 0 | Anti (740-790): 0 | H₂PO₄⁻ (≥790): 0',
            transform=self.ax.transAxes,
            fontsize=11,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8)
        )
        
        self.ax.legend(fontsize=9, loc='upper right')
    
    def _setup_buttons(self):
        """Create control buttons."""
        ax_done = self.fig.add_axes([0.3, 0.05, 0.15, 0.06])
        ax_reset = self.fig.add_axes([0.5, 0.05, 0.15, 0.06])
        ax_skip = self.fig.add_axes([0.7, 0.05, 0.15, 0.06])
        
        self.btn_done = Button(
            ax_done, 'Done', color='lightgreen', hovercolor='green'
        )
        self.btn_reset = Button(
            ax_reset, 'Reset', color='lightyellow', hovercolor='yellow'
        )
        self.btn_skip = Button(
            ax_skip, 'Skip Spectrum', color='lightcoral', hovercolor='red'
        )
        
        self.btn_done.on_clicked(self._on_done)
        self.btn_reset.on_clicked(self._on_reset)
        self.btn_skip.on_clicked(self._on_skip)
    
    def _connect_events(self):
        """Connect mouse click event handler."""
        self.cid = self.fig.canvas.mpl_connect(
            'button_press_event', self._on_click
        )
    
    def _on_click(self, event):
        """
        Handle mouse click events for peak selection.
        
        Args:
            event: Matplotlib mouse event
        """
        if event.inaxes != self.ax or event.button != 1:
            return
        
        # Find nearest data point
        idx = np.argmin(np.abs(self.x - event.xdata))
        

        x_peak = self.x[idx]
        y_peak = self.y[idx]
        
        self.peaks.append((x_peak, y_peak))
        self._update_plot()
    
    def _update_plot(self):
        """Update plot with current peak selections."""
        if len(self.peaks) > 0:
            peak_x = [p[0] for p in self.peaks]
            peak_y = [p[1] for p in self.peaks]
            self.peak_scatter.set_offsets(np.c_[peak_x, peak_y])
            
            # Clear old labels
            for label in self.peak_labels:
                label.remove()
            self.peak_labels = []
            
            # Count peaks by type
            n_gauche = sum(1 for p in self.peaks if p[0] < GAUCHE_THRESHOLD)
            n_anti = sum(1 for p in self.peaks 
                        if GAUCHE_THRESHOLD <= p[0] < PHOSPHATE_THRESHOLD)
            n_phosphate = sum(1 for p in self.peaks if p[0] >= PHOSPHATE_THRESHOLD)
            
            # Add labels to peaks
            for i, (x, y) in enumerate(self.peaks):
                if x < GAUCHE_THRESHOLD:
                    conformer = 'G'
                    color = 'blue'
                elif x < PHOSPHATE_THRESHOLD:
                    conformer = 'A'
                    color = 'green'
                else:
                    conformer = 'P'
                    color = 'orange'
                
                label = self.ax.text(
                    x, y,
                    f'  {conformer}{i+1}',
                    fontsize=10,
                    fontweight='bold',
                    color=color,
                    verticalalignment='bottom'
                )
                self.peak_labels.append(label)
        else:
            self.peak_scatter.set_offsets(np.empty((0, 2)))
            n_gauche = n_anti = n_phosphate = 0
        
        # Update instruction text
        self.instruction_text.set_text(
            f'Click on peak maxima\n'
            f'Peaks selected: {len(self.peaks)}\n'
            f'Gauche (<740): {n_gauche} | Anti (740-790): {n_anti} | '
            f'H₂PO₄⁻ (≥790): {n_phosphate}'
        )
        
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
    
    def _on_done(self, event):
        """Handle 'Done' button click."""
        if len(self.peaks) < 1:
            print("  ⚠ Warning: No peaks selected! "
                  "Please select at least 1 peak or skip.")
            return
        self.done = True
        self._cleanup()
    
    def _on_reset(self, event):
        """Handle 'Reset' button click."""
        self.peaks = []
        self._update_plot()
    
    def _on_skip(self, event):
        """Handle 'Skip Spectrum' button click."""
        self.skip = True
        self._cleanup()
    
    def _cleanup(self):
        """Clean up figure and disconnect events."""
        try:
            self.fig.canvas.mpl_disconnect(self.cid)
            plt.close(self.fig)
            plt.pause(0.01)
        except Exception:
            pass
    
    def pick_peaks(self) -> Optional[List[Tuple[float, float]]]:
        """
        Display picker and return selected peaks.
        
        Returns:
            List of (x, y) tuples for selected peaks, or None if skipped.
        """
        plt.show(block=True)
        
        if self.skip:
            return None
        
        if self.done and len(self.peaks) > 0:
            self.peaks.sort(key=lambda p: p[0])
            return self.peaks
        
        return None


def pick_peaks_interactive(
    x: np.ndarray,
    y: np.ndarray,
    temperature: float,
    filename: str
) -> Optional[List[Tuple[float, float]]]:
    """
    Wrapper function for interactive peak picking.
    
    Args:
        x: Wavenumber array
        y: Intensity array
        temperature: Measurement temperature in °C
        filename: Source filename
        
    Returns:
        List of selected peaks as (x, y) tuples, or None if skipped.
    """
    picker = InteractivePeakPicker(x, y, temperature, filename)
    peaks = picker.pick_peaks()
    plt.close('all')
    return peaks


# =============================================================================
# PEAK FITTING FUNCTIONS
# =============================================================================

def gaussian(x: np.ndarray, amplitude: float, center: float, 
            width: float) -> np.ndarray:
    """
    Calculate Gaussian peak profile.
    
    Args:
        x: Wavenumber array
        amplitude: Peak amplitude
        center: Peak center position
        width: Peak width (standard deviation)
        
    Returns:
        Gaussian profile values
    """
    return amplitude * np.exp(-((x - center) ** 2) / (2 * width ** 2))


def lorentzian(x: np.ndarray, amplitude: float, center: float,
              width: float) -> np.ndarray:
    """
    Calculate Lorentzian peak profile.
    
    Args:
        x: Wavenumber array
        amplitude: Peak amplitude
        center: Peak center position
        width: Peak width (FWHM/2)
        
    Returns:
        Lorentzian profile values
    """
    return amplitude * (width ** 2) / ((x - center) ** 2 + width ** 2)


def pseudo_voigt(x: np.ndarray, amplitude: float, center: float,
                width: float, fraction: float) -> np.ndarray:
    """
    Calculate pseudo-Voigt peak profile.
    
    Pseudo-Voigt is a weighted sum of Gaussian and Lorentzian profiles,
    commonly used to model Raman peaks.
    
    Args:
        x: Wavenumber array
        amplitude: Peak amplitude
        center: Peak center position
        width: Peak width
        fraction: Gaussian fraction (0=pure Lorentzian, 1=pure Gaussian)
        
    Returns:
        Pseudo-Voigt profile values
    """
    gauss = gaussian(x, amplitude, center, width)
    lorentz = lorentzian(x, amplitude, center, width)
    return fraction * gauss + (1 - fraction) * lorentz


def multi_peak_model(x: np.ndarray, peak_params: List[Tuple[float, float, float]],
                    fraction: float, baseline: float) -> np.ndarray:
    """
    Calculate multi-peak model with baseline.
    
    Args:
        x: Wavenumber array
        peak_params: List of (amplitude, center, width) tuples for each peak
        fraction: Gaussian/Lorentzian fraction
        baseline: Constant baseline offset
        
    Returns:
        Combined model values
    """
    total = np.ones_like(x) * baseline
    for amp, center, width in peak_params:
        total += pseudo_voigt(x, amp, center, width, fraction)
    return total


def fit_multiple_peaks(
    x: np.ndarray,
    y: np.ndarray,
    outlier_mask: np.ndarray,
    picked_peaks: List[Tuple[float, float]],
    fraction: float
) -> Tuple[Parameters, Dict]:
    """
    Fit multiple peaks to spectrum data, excluding outlier positions.
    
    Uses lmfit library to perform non-linear least squares fitting
    of pseudo-Voigt profiles to the spectrum.
    
    Args:
        x: Wavenumber array
        y: Intensity array
        outlier_mask: Boolean array indicating outlier positions to exclude
        picked_peaks: List of (x_pos, y_pos) tuples from peak picking
        fraction: Fixed Gaussian/Lorentzian fraction
        
    Returns:
        Tuple of (fitted parameters object, results dictionary)
        Results dict contains:
            - y_fit: Fitted model values
            - individual_peaks: List of individual peak profiles
            - baseline: Fitted baseline value
            - n_peaks: Number of peaks
            - residual: Fit residuals
            - chi_square: Chi-square value
            - reduced_chi_square: Reduced chi-square
            - success: Boolean fit success flag
    """
    # Exclude outlier positions from fitting
    valid_mask = ~outlier_mask
    x_fit = x[valid_mask]
    y_fit = y[valid_mask]
    
    params = Parameters()
    n_peaks = len(picked_peaks)
    
    # Setup parameters for each peak
    for i, (x_pos, y_pos) in enumerate(picked_peaks):
        params.add(
            f'amp{i}',
            value=y_pos,
            min=0,
            max=y_pos * AMPLITUDE_MAX_FACTOR
        )
        params.add(
            f'center{i}',
            value=x_pos,
            min=x_pos - PEAK_POSITION_TOLERANCE,
            max=x_pos + PEAK_POSITION_TOLERANCE
        )
        params.add(
            f'width{i}',
            value=INITIAL_PEAK_WIDTH,
            min=PEAK_WIDTH_MIN,
            max=PEAK_WIDTH_MAX
        )
    
    # Baseline and fraction parameters
    params.add(
        'baseline',
        value=0,
        min=y_fit.min(),
        max=y_fit.max() * BASELINE_MAX_FACTOR
    )
    params.add('fraction', value=fraction, vary=False)
    
    def residual(params, x, y, n_peaks):
        """Calculate residuals for fitting."""
        peak_params = []
        for i in range(n_peaks):
            peak_params.append((
                params[f'amp{i}'],
                params[f'center{i}'],
                params[f'width{i}']
            ))
        model = multi_peak_model(x, peak_params, params['fraction'], 
                                params['baseline'])
        return y - model
    
    # Perform fit on valid (non-outlier) data only
    fit_result = lmfit_minimize(
        residual, params, args=(x_fit, y_fit, n_peaks), method='leastsq'
    )
    
    # Extract fitted parameters
    peak_params_fitted = []
    for i in range(n_peaks):
        peak_params_fitted.append((
            fit_result.params[f'amp{i}'].value,
            fit_result.params[f'center{i}'].value,
            fit_result.params[f'width{i}'].value
        ))
    
    baseline_fitted = fit_result.params['baseline'].value
    
    # Calculate fit for entire spectrum (including outlier positions for visualization)
    y_fit_full = multi_peak_model(x, peak_params_fitted, fraction, baseline_fitted)
    
    # Calculate individual peaks
    individual_peaks = []
    for amp, center, width in peak_params_fitted:
        peak = pseudo_voigt(x, amp, center, width, fraction)
        individual_peaks.append(peak)
    
    # Calculate residual only for valid points
    residual_full = np.zeros_like(y)
    residual_full[valid_mask] = y_fit - multi_peak_model(
        x_fit, peak_params_fitted, fraction, baseline_fitted
    )
    residual_full[outlier_mask] = np.nan  # Mark outlier positions as NaN in residual
    
    return fit_result.params, {
        'y_fit': y_fit_full,
        'individual_peaks': individual_peaks,
        'baseline': baseline_fitted,
        'n_peaks': n_peaks,
        'residual': residual_full,
        'chi_square': fit_result.chisqr,
        'reduced_chi_square': fit_result.redchi,
        'success': fit_result.success
    }


# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def extract_temperature(filename: str) -> Optional[float]:
    """
    Extract temperature from filename.
    
    Searches for pattern like '25C_', '-40C_', etc. in filename.
    
    Args:
        filename: Filename to parse
        
    Returns:
        Temperature in °C, or None if not found
    """
    match = re.search(r'(-?\d+)C', filename)
    if match:
        return float(match.group(1))
    return None


def calculate_peak_area(x: np.ndarray, peak: np.ndarray) -> float:
    """
    Calculate area under peak using Simpson's rule.
    
    Args:
        x: Wavenumber array
        peak: Peak intensity values
        
    Returns:
        Integrated peak area
    """
    return simpson(peak, x=x)


def classify_peak_type(center: float) -> str:
    """
    Classify peak as gauche, anti, or phosphate based on position.
    
    Args:
        center: Peak center position in cm⁻¹
        
    Returns:
        Peak type: 'gauche', 'anti', or 'phosphate'
    """
    if center < GAUCHE_THRESHOLD:
        return 'gauche'
    elif center < PHOSPHATE_THRESHOLD:
        return 'anti'
    else:
        return 'phosphate'


def get_peak_color(center: float) -> str:
    """
    Get display color for peak based on type.
    
    Args:
        center: Peak center position in cm⁻¹
        
    Returns:
        Color name: 'blue' (gauche), 'green' (anti), or 'orange' (phosphate)
    """
    if center < GAUCHE_THRESHOLD:
        return 'blue'
    elif center < PHOSPHATE_THRESHOLD:
        return 'green'
    else:
        return 'orange'


def cleanup_gui_resources():
    """Clean up lingering GUI windows and resources."""
    # Close all matplotlib figures
    plt.close('all')
    
    # Aggressive tkinter cleanup
    try:
        # Find and destroy all Tk instances
        root_list = []
        for obj in gc.get_objects():
            try:
                if isinstance(obj, tk.Tk) or isinstance(obj, tk.Toplevel):
                    root_list.append(obj)
            except Exception:
                pass
        
        for root in root_list:
            try:
                root.quit()
            except Exception:
                pass
            try:
                root.destroy()
            except Exception:
                pass
        
        del root_list
    except Exception:
        pass
    
    # Multiple garbage collection passes
    for _ in range(3):
        gc.collect()
        time.sleep(0.1)


# =============================================================================
# DATA PROCESSING FUNCTIONS
# =============================================================================

def load_and_correct_spectra(
    data_folder: str
) -> Tuple[List[np.ndarray], List[np.ndarray], List[float], List[str]]:
    """
    Load raw data files and perform baseline correction.
    
    Note: Outlier detection is now performed AFTER fitting (two-round approach).
    
    Args:
        data_folder: Path to folder containing .txt data files
        
    Returns:
        Tuple of (x_arrays, y_corrected_arrays, temperatures, filenames)
    """
    print("\n" + "="*80)
    print("STEP 1: BASELINE CORRECTION")
    print("="*80)
    
    raw_files = [
        f for f in os.listdir(data_folder)
        if os.path.isfile(os.path.join(data_folder, f))
        and not f.startswith('.')
        and f.endswith('.txt')
    ]
    
    print(f"\nFound {len(raw_files)} spectra")
    
    all_x = []
    all_y_corrected = []
    all_temperatures = []
    valid_files = []
    
    for filename in raw_files:
        try:
            filepath = os.path.join(data_folder, filename)
            data = np.loadtxt(filepath)
            x_full = data[:, 0]
            y_full = data[:, 1]
            
            temp = extract_temperature(filename)
            if temp is None:
                print(f"  ⚠ Skipped {filename}: no temperature found")
                continue
            
            # Apply baseline correction
            y_corrected, _ = baseline_correct_with_linear_shift(
                x_full, y_full,
                method='arPLS',
                lam=1e6,
                ratio=0.01,
                percentile_threshold=25,
                target_min=0,
                use_linear=True
            )
            
            all_x.append(x_full)
            all_y_corrected.append(y_corrected)
            all_temperatures.append(temp)
            valid_files.append(filename)
            
            print(f"  ✓ {temp:3.0f}°C - {filename}")
            
        except Exception as e:
            print(f"  ✗ Error with {filename}: {e}")
    
    print(f"\n✓ Successfully processed {len(valid_files)} spectra")
    
    if len(valid_files) == 0:
        print("\n✗ ERROR: No spectra were successfully loaded!")
        raise SystemExit(1)
    
    # Sort by temperature
    sort_indices = np.argsort(all_temperatures)
    all_x = [all_x[i] for i in sort_indices]
    all_y_corrected = [all_y_corrected[i] for i in sort_indices]
    all_temperatures = [all_temperatures[i] for i in sort_indices]
    valid_files = [valid_files[i] for i in sort_indices]
    
    print(f"Temperature range: {min(all_temperatures):.0f}°C to "
          f"{max(all_temperatures):.0f}°C")
    
    return (all_x, all_y_corrected, all_temperatures, valid_files)


def filter_by_temperature(
    data: Tuple[List, List, List, List],
    temp_min: Optional[float],
    temp_max: Optional[float]
) -> Tuple[List, List, List, List]:
    """
    Filter data by temperature range.
    
    Args:
        data: Tuple of (x_arrays, y_arrays, temperatures, filenames)
        temp_min: Minimum temperature (None for no filtering)
        temp_max: Maximum temperature (None for no filtering)
        
    Returns:
        Filtered data tuple
    """
    all_x, all_y, all_temps, all_files = data
    
    if temp_min is None or temp_max is None:
        print("\n✓ Using all data (no temperature filtering)")
        return data
    
    print("\n" + "="*80)
    print("FILTERING DATA BY TEMPERATURE RANGE")
    print("="*80)
    print(f"Keeping only data between {temp_min:.0f}°C and {temp_max:.0f}°C")
    
    indices_to_keep = [
        i for i, temp in enumerate(all_temps)
        if temp_min <= temp <= temp_max
    ]
    
    if len(indices_to_keep) == 0:
        print(f"\n✗ ERROR: No data points within specified range!")
        print(f"   Data range: {min(all_temps):.0f}°C to "
              f"{max(all_temps):.0f}°C")
        print(f"   Filter range: {temp_min:.0f}°C to {temp_max:.0f}°C")
        raise SystemExit(1)
    
    filtered_x = [all_x[i] for i in indices_to_keep]
    filtered_y = [all_y[i] for i in indices_to_keep]
    filtered_temps = [all_temps[i] for i in indices_to_keep]
    filtered_files = [all_files[i] for i in indices_to_keep]
    
    print(f"\n✓ Filtered from {len(all_temps)} to {len(indices_to_keep)} spectra")
    print(f"  New temperature range: {min(filtered_temps):.0f}°C to "
          f"{max(filtered_temps):.0f}°C")
    
    return (filtered_x, filtered_y, filtered_temps, filtered_files)


def perform_interactive_peak_picking(
    all_x: List[np.ndarray],
    all_y_corrected: List[np.ndarray],
    all_temperatures: List[float],
    valid_files: List[str]
) -> Tuple[List[np.ndarray], List[np.ndarray], List[np.ndarray], List[np.ndarray],
           List[List[Tuple]], List[np.ndarray], List[np.ndarray], List[float], List[str]]:
    """
    Perform interactive peak picking for all spectra.
    
    Args:
        all_x: List of wavenumber arrays
        all_y_corrected: List of corrected intensity arrays
        all_temperatures: List of temperatures
        valid_files: List of filenames
        
    Returns:
        Tuple of (peak_region_x, peak_region_y, peak_region_y_original,
                 peak_region_outliers, all_picked_peaks, filtered_x, filtered_y,
                 filtered_temps, filtered_files)
    """
    print("\n" + "="*80)
    print("STEP 2: INTERACTIVE PEAK PICKING")
    print("="*80)
    print("For each spectrum, a POP-UP window will appear.")
    print("Click on the peak maxima you want to fit.")
    print("Use the buttons: 'Done' to finish, 'Reset' to clear, "
          "'Skip Spectrum' to exclude.")
    print("="*80)
    
    peak_region_x = []
    peak_region_y = []
    peak_region_y_original = []
    peak_region_outliers = []
    all_picked_peaks = []
    skipped_indices = []
    
    for i, (x, y, temp, filename) in enumerate(
        zip(all_x, all_y_corrected, all_temperatures, valid_files)
    ):
        # Extract region of interest
        mask = (x >= PEAK_REGION_MIN) & (x <= PEAK_REGION_MAX)
        x_roi = x[mask]
        y_roi = y[mask]
        
        print(f"\nSpectrum {i+1}/{len(all_x)}: {temp:.0f}°C")
        print(f"  → Opening interactive peak picker window...")
        
        picked_peaks = pick_peaks_interactive(
            x_roi, y_roi, temp, filename
        )
        
        if picked_peaks is None:
            print(f"  ⊗ Skipped spectrum at {temp:.0f}°C")
            skipped_indices.append(i)
            continue
        
        # Count peaks by type
        gauche_peaks = [p for p in picked_peaks if p[0] < GAUCHE_THRESHOLD]
        anti_peaks = [p for p in picked_peaks 
                     if GAUCHE_THRESHOLD <= p[0] < PHOSPHATE_THRESHOLD]
        phosphate_peaks = [p for p in picked_peaks if p[0] >= PHOSPHATE_THRESHOLD]
        
        print(f"  ✓ Selected {len(picked_peaks)} peaks:")
        print(f"    - Gauche (<740 cm⁻¹): {len(gauche_peaks)} peaks")
        print(f"    - Anti (740-790 cm⁻¹): {len(anti_peaks)} peaks")
        print(f"    - H₂PO₄⁻ (≥790 cm⁻¹): {len(phosphate_peaks)} peaks")
        
        peak_region_x.append(x_roi)
        peak_region_y.append(y_roi)
        peak_region_y_original.append(y_roi)  # Same as y_roi now
        peak_region_outliers.append(np.zeros(len(x_roi), dtype=bool))  # No pre-fit outliers
        all_picked_peaks.append(picked_peaks)
    
    # Remove skipped spectra
    if len(skipped_indices) > 0:
        print(f"\n✓ Skipped {len(skipped_indices)} spectra")
        for idx in reversed(skipped_indices):
            del all_x[idx]
            del all_y_corrected[idx]
            del all_temperatures[idx]
            del valid_files[idx]
    
    print(f"\n✓ Peak picking complete for {len(all_picked_peaks)} spectra")
    
    return (peak_region_x, peak_region_y, peak_region_y_original, 
            peak_region_outliers, all_picked_peaks, all_x, all_y_corrected,
            all_temperatures, valid_files)


def optimize_voigt_fraction(
    peak_region_x: List[np.ndarray],
    peak_region_y: List[np.ndarray],
    peak_region_outliers: List[np.ndarray],
    all_picked_peaks: List[List[Tuple]]
) -> float:
    """
    Optimize Gaussian/Lorentzian fraction for best fit.
    
    Tests multiple fraction values and selects the one giving
    lowest average chi-square.
    
    Args:
        peak_region_x: List of wavenumber arrays
        peak_region_y: List of intensity arrays
        peak_region_outliers: List of outlier mask arrays
        all_picked_peaks: List of picked peaks for each spectrum
        
    Returns:
        Optimal fraction value (0=pure Lorentzian, 1=pure Gaussian)
    """
    print("\n" + "="*80)
    print("STEP 3: OPTIMIZE GAUSSIAN/LORENTZIAN FRACTION")
    print("="*80)
    
    test_fractions = [0.0, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0]
    fraction_results = []
    
    for frac in test_fractions:
        total_chi = 0
        n_success = 0
        
        for x, y, outliers, peaks in zip(
            peak_region_x, peak_region_y, peak_region_outliers, all_picked_peaks
        ):
            try:
                _, result = fit_multiple_peaks(x, y, outliers, peaks, fraction=frac)
                if result['success']:
                    total_chi += result['chi_square']
                    n_success += 1
            except Exception:
                pass
        
        avg_chi = total_chi / n_success if n_success > 0 else np.inf
        fraction_results.append({
            'fraction': frac,
            'avg_chi_square': avg_chi,
            'n_success': n_success
        })
        
        print(f"  Fraction {frac:.1f} ({frac*100:3.0f}% Gauss): "
              f"χ²={avg_chi:.2e}, Success={n_success}/{len(peak_region_x)}")
    
    # Find optimal
    fraction_df = pd.DataFrame(fraction_results)
    optimal_idx = fraction_df['avg_chi_square'].idxmin()
    optimal_fraction = fraction_df.loc[optimal_idx, 'fraction']
    
    print(f"\n✓ OPTIMAL FRACTION: {optimal_fraction:.2f}")
    print(f"  ({optimal_fraction*100:.0f}% Gaussian, "
          f"{(1-optimal_fraction)*100:.0f}% Lorentzian)")
    
    return optimal_fraction




def fit_spectrum_two_rounds(
    x: np.ndarray,
    y: np.ndarray,
    picked_peaks: List[Tuple[float, float]],
    fraction: float,
    temperature: float,
    filename: str,
    data_folder: str
) -> Tuple[Parameters, Dict, np.ndarray]:
    """
    Fit spectrum using two-round approach with outlier detection.
    
    Round 1: Fit all data points
    Detect outliers: Based on residuals from round 1 fit
    Round 2: Refit without outliers using SAME initial peak positions
    
    Args:
        x: Wavenumber array
        y: Intensity array
        picked_peaks: List of (position, height) tuples from peak picking
        fraction: Gaussian/Lorentzian fraction
        temperature: Measurement temperature
        filename: Source filename
        data_folder: Output folder for plots
        
    Returns:
        Tuple of (fitted_params, result_dict, outlier_mask)
    """
    # ROUND 1: Fit all data (no outliers excluded)
    no_outliers = np.zeros(len(x), dtype=bool)
    params_round1, result_round1 = fit_multiple_peaks(
        x, y, no_outliers, picked_peaks, fraction
    )
    
    # Detect outliers from round 1 residuals (uses OUTLIER_THRESHOLD_SIGMA constant)
    y_fitted_round1 = result_round1['y_fit']
    outlier_mask = detect_outliers_from_residuals(y, y_fitted_round1)
    
    n_outliers = np.sum(outlier_mask)
    
    # Create outlier detection plot
    plot_outlier_detection(
        x, y, y_fitted_round1, outlier_mask,
        temperature, filename, data_folder
    )
    
    if n_outliers == 0:
        # No outliers detected, use round 1 result
        return params_round1, result_round1, outlier_mask
    
    # ROUND 2: Refit without outliers, using SAME initial peak positions
    params_round2, result_round2 = fit_multiple_peaks(
        x, y, outlier_mask, picked_peaks, fraction
    )
    
    return params_round2, result_round2, outlier_mask

def fit_all_spectra(
    peak_region_x: List[np.ndarray],
    peak_region_y: List[np.ndarray],
    peak_region_y_original: List[np.ndarray],
    peak_region_outliers: List[np.ndarray],
    all_temperatures: List[float],
    valid_files: List[str],
    all_picked_peaks: List[List[Tuple]],
    optimal_fraction: float,
    data_folder: str
) -> Tuple[pd.DataFrame, List[Dict]]:
    """
    Fit all spectra with optimal fraction and calculate conformer ratios.
    
    Args:
        peak_region_x: List of wavenumber arrays
        peak_region_y: List of intensity arrays (cleaned)
        peak_region_y_original: List of original intensity arrays (before interpolation)
        peak_region_outliers: List of outlier mask arrays
        all_temperatures: List of temperatures
        valid_files: List of filenames
        all_picked_peaks: List of picked peaks
        optimal_fraction: Optimal Gaussian/Lorentzian fraction
        
    Returns:
        Tuple of (results DataFrame, list of fit parameter dictionaries)
    """
    print("\n" + "="*80)
    print("STEP 4: FIT ALL SPECTRA AND CALCULATE CONFORMER RATIOS")
    print("="*80)
    
    all_fit_results = []
    all_fit_params = []
    
    for i, (x, y, y_orig, outliers, temp, filename, peaks) in enumerate(
        zip(peak_region_x, peak_region_y, peak_region_y_original, peak_region_outliers,
            all_temperatures, valid_files, all_picked_peaks)
    ):
        try:
            params, result, outlier_mask = fit_spectrum_two_rounds(
                x, y, peaks, optimal_fraction, temp, filename, data_folder
            )
            
            # Calculate peak areas by type
            gauche_area = 0
            anti_area = 0
            phosphate_area = 0
            peak_areas = []
            
            for j, peak in enumerate(result['individual_peaks']):
                area = calculate_peak_area(x, peak)
                peak_areas.append(area)
                
                center = params[f'center{j}'].value
                peak_type = classify_peak_type(center)
                
                if peak_type == 'gauche':
                    gauche_area += area
                elif peak_type == 'anti':
                    anti_area += area
                else:
                    phosphate_area += area
            
            total_area = sum(peak_areas)
            
            # Calculate conformer ratio (excluding phosphate)
            conformer_ratio = (
                gauche_area / (gauche_area + anti_area)
                if (gauche_area + anti_area) > 0 else 0
            )
            
            # Store results
            result_dict = {
                'temperature': temp,
                'filename': filename,
                'n_peaks': result['n_peaks'],
                'n_outliers_excluded': np.sum(outliers),
                'chi_square': result['chi_square'],
                'reduced_chi_square': result['reduced_chi_square'],
                'baseline': result['baseline'],
                'total_area': total_area,
                'gauche_area': gauche_area,
                'anti_area': anti_area,
                'phosphate_area': phosphate_area,
                'conformer_ratio': conformer_ratio
            }
            
            # Add individual peak parameters
            for j in range(result['n_peaks']):
                center = params[f'center{j}'].value
                conformer_type = classify_peak_type(center)
                
                result_dict[f'peak{j+1}_center'] = center
                result_dict[f'peak{j+1}_amp'] = params[f'amp{j}'].value
                result_dict[f'peak{j+1}_width'] = params[f'width{j}'].value
                result_dict[f'peak{j+1}_area'] = peak_areas[j]
                result_dict[f'peak{j+1}_area_fraction'] = (
                    peak_areas[j] / total_area if total_area > 0 else 0
                )
                result_dict[f'peak{j+1}_type'] = conformer_type
            
            all_fit_results.append(result_dict)
            
            all_fit_params.append({
                'x': x,
                'y': y,
                'y_original': y_orig,  # Store original values for plotting
                'outlier_mask': outlier_mask,
                'y_fit': result['y_fit'],
                'individual_peaks': result['individual_peaks'],
                'baseline': result['baseline'],
                'residual': result['residual'],
                'n_peaks': result['n_peaks'],
                'params': params
            })
            
            outlier_info = f", {np.sum(outlier_mask)} outliers excluded" if np.sum(outlier_mask) > 0 else ""
            print(f"  ✓ {temp:3.0f}°C: χ²={result['chi_square']:.2e}, "
                  f"Ratio={conformer_ratio:.3f} "
                  f"(G={gauche_area:.0f}, A={anti_area:.0f}, "
                  f"P={phosphate_area:.0f}), BL={result['baseline']:.1f}{outlier_info}")
            
        except Exception as e:
            print(f"  ✗ {temp:3.0f}°C: {e}")
    
    results_df = pd.DataFrame(all_fit_results)
    print(f"\n✓ Successfully fitted {len(results_df)} spectra")
    
    return results_df, all_fit_params


def filter_duplicate_temperatures(
    results_df: pd.DataFrame,
    all_fit_params: List[Dict],
    peak_region_x: List[np.ndarray],
    peak_region_y: List[np.ndarray],
    peak_region_y_original: List[np.ndarray]
) -> Tuple[pd.DataFrame, List[Dict], List[np.ndarray], List[np.ndarray], List[np.ndarray]]:
    """
    Filter duplicate temperatures, keeping only best fit for each.
    
    Args:
        results_df: Results DataFrame
        all_fit_params: List of fit parameters
        peak_region_x: List of wavenumber arrays
        peak_region_y: List of intensity arrays
        peak_region_y_original: List of original intensity arrays
        
    Returns:
        Filtered data tuple
    """
    print("\n" + "="*80)
    print("FILTERING DUPLICATE TEMPERATURES")
    print("="*80)
    
    temp_counts = results_df['temperature'].value_counts()
    duplicates = temp_counts[temp_counts > 1]
    
    if len(duplicates) > 0:
        print(f"Found {len(duplicates)} temperatures with multiple measurements:")
        for temp, count in duplicates.items():
            print(f"  {temp:.0f}°C: {count} measurements")
        
        best_indices = results_df.groupby('temperature')['chi_square'].idxmin()
        
        filtered_results_df = results_df.loc[best_indices].reset_index(drop=True)
        filtered_fit_params = [all_fit_params[i] for i in best_indices]
        filtered_peak_region_x = [peak_region_x[i] for i in best_indices]
        filtered_peak_region_y = [peak_region_y[i] for i in best_indices]
        filtered_peak_region_y_original = [peak_region_y_original[i] for i in best_indices]
        
        print(f"\n✓ Filtered from {len(results_df)} to "
              f"{len(filtered_results_df)} spectra")
        print("  (kept best fit for each temperature)")
        
        return (filtered_results_df, filtered_fit_params,
                filtered_peak_region_x, filtered_peak_region_y,
                filtered_peak_region_y_original)
    else:
        print("No duplicate temperatures found - using all spectra")
        return (results_df, all_fit_params, peak_region_x, peak_region_y,
                peak_region_y_original)


def normalize_spectra(
    peak_region_x: List[np.ndarray],
    peak_region_y: List[np.ndarray],
    results_df: pd.DataFrame
) -> List[np.ndarray]:
    """
    Normalize spectra by conformer peak areas (excluding phosphate).
    
    Args:
        peak_region_x: List of wavenumber arrays
        peak_region_y: List of intensity arrays
        results_df: Results DataFrame
        
    Returns:
        List of normalized intensity arrays
    """
    print("\n" + "="*80)
    print("STEP 5: NORMALIZE SPECTRA BY SUM OF CONFORMER PEAK AREAS")
    print("="*80)
    
    all_y_normalized = []
    
    for i, (x, y) in enumerate(zip(peak_region_x, peak_region_y)):
        conformer_peak_area = (
            results_df.loc[i, 'gauche_area'] + results_df.loc[i, 'anti_area']
        )
        
        y_norm = y / conformer_peak_area if conformer_peak_area != 0 else y
        all_y_normalized.append(y_norm)
        
        temp = results_df.loc[i, 'temperature']
        print(f"  ✓ {temp:3.0f}°C: Conformer peak area (G+A) = "
              f"{conformer_peak_area:.1f}")
    
    print(f"\n✓ Normalized {len(all_y_normalized)} spectra by conformer peak "
          f"areas (excluding phosphate)")
    
    return all_y_normalized


# =============================================================================
# PLOTTING FUNCTIONS
# =============================================================================

def create_temperature_colormap(
    temperatures: np.ndarray,
    temp_min: Optional[float],
    temp_max: Optional[float]
) -> Tuple[List, float, float]:
    """
    Create color mapping for temperature-resolved plots.
    
    Args:
        temperatures: Array of measurement temperatures
        temp_min: Custom minimum for color scale (None for auto)
        temp_max: Custom maximum for color scale (None for auto)
        
    Returns:
        Tuple of (color list, temp_range_min, temp_range_max)
    """
    print("\n" + "="*80)
    print("OUTPUT 1: TEMPERATURE-RESOLVED SPECTRA")
    print("="*80)
    
    if temp_min is not None and temp_max is not None:
        temp_range_min = temp_min
        temp_range_max = temp_max
        print(f"Using custom color range: {temp_range_min:.0f} to "
              f"{temp_range_max:.0f} °C")
    else:
        temp_range_min = temperatures.min()
        temp_range_max = temperatures.max()
        print(f"Using automatic color range: {temp_range_min:.0f} to "
              f"{temp_range_max:.0f} °C")
    
    colors = []
    for temp in temperatures:
        temp_norm = (temp - temp_range_min) / (temp_range_max - temp_range_min)
        temp_norm = np.clip(temp_norm, 0, 1)
        colors.append(cm.Spectral_r(temp_norm))
    
    return colors, temp_range_min, temp_range_max


def plot_temperature_spectra(
    peak_region_x: List[np.ndarray],
    peak_region_y: List[np.ndarray],
    all_y_normalized: List[np.ndarray],
    temperatures: np.ndarray,
    colors: List,
    data_folder: str,
    all_fit_params: List[Dict] = None
):
    """
    Create temperature-resolved spectral plots.
    
    Args:
        peak_region_x: List of wavenumber arrays
        peak_region_y: List of raw intensity arrays
        all_y_normalized: List of normalized intensity arrays
        temperatures: Array of temperatures
        colors: List of colors for each temperature
        data_folder: Output folder path
        all_fit_params: List of fit parameters containing outlier masks (optional)
    """
    # Remove outliers from spectra for clean plotting
    if all_fit_params is not None:
        peak_region_y_clean = []
        all_y_normalized_clean = []
        
        for x, y_raw, y_norm, fit_params in zip(peak_region_x, peak_region_y, all_y_normalized, all_fit_params):
            outlier_mask = fit_params['outlier_mask']
            
            if np.any(outlier_mask):
                # Interpolate outliers using surrounding valid points
                valid_mask = ~outlier_mask
                
                # Linear interpolation for raw data
                y_raw_clean = y_raw.copy()
                y_raw_clean[outlier_mask] = np.interp(
                    x[outlier_mask],  # x positions of outliers
                    x[valid_mask],    # x positions of valid points
                    y_raw[valid_mask] # y values of valid points
                )
                
                # Linear interpolation for normalized data
                y_norm_clean = y_norm.copy()
                y_norm_clean[outlier_mask] = np.interp(
                    x[outlier_mask],
                    x[valid_mask],
                    y_norm[valid_mask]
                )
            else:
                # No outliers, use original data
                y_raw_clean = y_raw
                y_norm_clean = y_norm
            
            peak_region_y_clean.append(y_raw_clean)
            all_y_normalized_clean.append(y_norm_clean)
    else:
        # No outlier removal if fit_params not provided
        peak_region_y_clean = peak_region_y
        all_y_normalized_clean = all_y_normalized
    
    # 1A: Normalized spectra
    fig, ax = plt.subplots(figsize=(10, 7))
    
    for i, (x, y_norm, temp, color) in enumerate(
        zip(peak_region_x, all_y_normalized_clean, temperatures, colors)
    ):
        ax.plot(x, y_norm, color=color, linewidth=2, alpha=0.8,
               label=f'{temp:.0f} °C')
    
    ax.set_xlabel('wavenumber / cm$^{-1}$', fontsize=13, fontweight='bold')
    ax.set_ylabel('Raman intensity', fontsize=13, fontweight='bold')
    ax.set_xlim(PEAK_REGION_MIN, PEAK_REGION_MAX)
    ax.set_yticks([])
    ax.tick_params(axis='both', which='both', direction='in',
                  top=True, right=True, labeltop=False, labelright=False)
    ax.legend(fontsize=9, loc='best', ncol=2, frameon=True,
             fancybox=False, edgecolor='black')
    
    plt.tight_layout()
    plt.savefig(
        os.path.join(data_folder, 'Output_1A_Temperature_Spectra_Normalized.png'),
        dpi=OUTPUT_DPI,
        bbox_inches='tight'
    )
    plt.close()
    
    print("✓ Saved: Output_1A_Temperature_Spectra_Normalized.png")
    
    # 1B: Non-normalized spectra
    fig, ax = plt.subplots(figsize=(10, 7))
    
    for i, (x, y, temp, color) in enumerate(
        zip(peak_region_x, peak_region_y_clean, temperatures, colors)
    ):
        ax.plot(x, y, color=color, linewidth=2, alpha=0.8,
               label=f'{temp:.0f} °C')
    
    ax.set_xlabel('wavenumber / cm$^{-1}$', fontsize=13, fontweight='bold')
    ax.set_ylabel('Raman intensity', fontsize=13, fontweight='bold')
    ax.set_xlim(PEAK_REGION_MIN, PEAK_REGION_MAX)
    ax.set_yticks([])
    ax.tick_params(axis='both', which='both', direction='in',
                  top=True, right=True, labeltop=False, labelright=False)
    ax.legend(fontsize=9, loc='best', ncol=2, frameon=True,
             fancybox=False, edgecolor='black')
    
    plt.tight_layout()
    plt.savefig(
        os.path.join(data_folder, 'Output_1B_Temperature_Spectra_Raw.png'),
        dpi=OUTPUT_DPI,
        bbox_inches='tight'
    )
    plt.close()
    
    print("✓ Saved: Output_1B_Temperature_Spectra_Raw.png")


def plot_fit_example(
    fit_data: Dict,
    results: pd.Series,
    optimal_fraction: float,
    title: str,
    output_filename: str,
    data_folder: str
):
    """
    Create plot showing fitted spectrum with residuals.
    
    Args:
        fit_data: Dictionary with fit parameters and results
        results: Series with fit results for this spectrum
        optimal_fraction: Gaussian/Lorentzian fraction
        title: Plot title
        output_filename: Output filename
        data_folder: Output folder path
    """
    fig, axes = plt.subplots(
        2, 1, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1]}
    )
    
    # Create smooth fit curve
    x_smooth = np.linspace(
        fit_data['x'].min(), fit_data['x'].max(), len(fit_data['x']) * 10
    )
    
    peak_params = []
    for j in range(fit_data['n_peaks']):
        peak_params.append((
            results[f'peak{j+1}_amp'],
            results[f'peak{j+1}_center'],
            results[f'peak{j+1}_width']
        ))
    
    y_fit_smooth = multi_peak_model(
        x_smooth, peak_params, optimal_fraction, results['baseline']
    )
    
    # Main plot
    ax_main = axes[0]
    ax_main.plot(
        fit_data['x'], fit_data['y'], 'ko', markersize=5, alpha=0.6,
        label='Data', zorder=1
    )
    ax_main.plot(
        x_smooth, y_fit_smooth, '-', color='red', linewidth=2,
        label=f'Total Fit ({results["temperature"]:.0f} °C)', zorder=3
    )
    
    # Mark outlier positions with ORIGINAL values if any
    if 'outlier_mask' in fit_data and np.any(fit_data['outlier_mask']):
        ax_main.scatter(
            fit_data['x'][fit_data['outlier_mask']],
            fit_data['y_original'][fit_data['outlier_mask']],  # Use ORIGINAL values
            c='red', s=80, marker='x', linewidths=2,
            label=f'Excluded Outliers (n={np.sum(fit_data["outlier_mask"])})',
            zorder=7, alpha=0.7
        )
    
    # Baseline
    ax_main.axhline(
        y=fit_data['baseline'], color='gray', linestyle=':',
        linewidth=1.5, alpha=0.7, label='Baseline'
    )
    
    # Individual peaks
    for j, peak in enumerate(fit_data['individual_peaks']):
        center = results[f'peak{j+1}_center']
        peak_type = results[f'peak{j+1}_type']
        color = get_peak_color(center)
        
        ax_main.plot(
            fit_data['x'], peak + fit_data['baseline'],
            '--', color=color, linewidth=2.5, alpha=0.8,
            label=f'{peak_type.capitalize()} ({center:.1f} cm⁻¹)', zorder=2
        )
    
    # Statistics box
    ax_main.text(
        0.98, 0.98,
        f'χ² = {results["chi_square"]:.2e}\n'
        f'Ratio = {results["conformer_ratio"]:.3f}',
        transform=ax_main.transAxes,
        fontsize=11,
        verticalalignment='top',
        horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, 
                 edgecolor='black')
    )
    
    ax_main.set_ylabel('Raman intensity', fontsize=13, fontweight='bold')
    ax_main.set_title(title, fontsize=14, fontweight='bold')
    ax_main.legend(fontsize=9, loc='best', frameon=True,
                  fancybox=False, edgecolor='black')
    ax_main.set_xlim(PEAK_REGION_MIN, PEAK_REGION_MAX)
    ax_main.tick_params(axis='both', which='both', direction='in',
                       top=True, right=True, labeltop=False, labelright=False)
    
    # Residuals plot
    ax_resid = axes[1]
    
    # Only plot non-outlier residuals
    valid_residuals = ~np.isnan(fit_data['residual'])
    ax_resid.plot(
        fit_data['x'][valid_residuals], 
        fit_data['residual'][valid_residuals], 
        'ko-',
        markersize=3, linewidth=1, alpha=0.6
    )
    ax_resid.axhline(
        y=0, color='red', linestyle='--', linewidth=2, alpha=0.7
    )
    ax_resid.fill_between(
        fit_data['x'][valid_residuals], 0, 
        fit_data['residual'][valid_residuals], 
        alpha=0.3, color='gray'
    )
    ax_resid.set_xlabel('wavenumber / cm$^{-1}$', fontsize=13, fontweight='bold')
    ax_resid.set_ylabel('Residual', fontsize=11, fontweight='bold')
    ax_resid.set_xlim(PEAK_REGION_MIN, PEAK_REGION_MAX)
    ax_resid.tick_params(axis='both', which='both', direction='in',
                        top=True, right=True, labeltop=False, labelright=False)
    
    plt.tight_layout()
    plt.savefig(
        os.path.join(data_folder, output_filename),
        dpi=OUTPUT_DPI,
        bbox_inches='tight'
    )
    plt.close()
    
    print(f"✓ Saved: {output_filename} (Temperature: "
          f"{results['temperature']:.0f}°C)")


def plot_best_worst_fits(
    results_df: pd.DataFrame,
    all_fit_params: List[Dict],
    optimal_fraction: float,
    data_folder: str
):
    """
    Create plots for best and worst fitted spectra.
    
    Args:
        results_df: Results DataFrame
        all_fit_params: List of fit parameters
        optimal_fraction: Optimal Gaussian/Lorentzian fraction
        data_folder: Output folder path
    """
    print("\n" + "="*80)
    print("OUTPUT 2: BEST AND WORST FIT EXAMPLES")
    print("="*80)
    
    # Best fit
    best_idx = results_df['chi_square'].idxmin()
    plot_fit_example(
        all_fit_params[best_idx],
        results_df.loc[best_idx],
        optimal_fraction,
        'Best fitted spectrum',
        'Output_2A_Best_Fit.png',
        data_folder
    )
    
    # Worst fit
    worst_idx = results_df['chi_square'].idxmax()
    plot_fit_example(
        all_fit_params[worst_idx],
        results_df.loc[worst_idx],
        optimal_fraction,
        'Worst fitted spectrum',
        'Output_2B_Worst_Fit.png',
        data_folder
    )


def plot_conformer_ratio(
    results_df: pd.DataFrame,
    temp_range_min: float,
    temp_range_max: float,
    data_folder: str
):
    """
    Create conformer ratio vs temperature plot.
    
    Args:
        results_df: Results DataFrame
        temp_range_min: Minimum temperature for color scale
        temp_range_max: Maximum temperature for color scale
        data_folder: Output folder path
    """
    print("\n" + "="*80)
    print("OUTPUT 3: CONFORMER RATIO ANALYSIS")
    print("="*80)
    
    fig, ax = plt.subplots(figsize=(10, 7))
    
    # Get colors for each temperature
    colors_for_temps = []
    for temp in results_df['temperature']:
        temp_norm = (temp - temp_range_min) / (temp_range_max - temp_range_min)
        temp_norm = np.clip(temp_norm, 0, 1)
        colors_for_temps.append(cm.Spectral_r(temp_norm))
    
    # Plot data points
    for temp, ratio, color in zip(
        results_df['temperature'],
        results_df['conformer_ratio'],
        colors_for_temps
    ):
        ax.plot(
            temp, ratio, 'o', color=color, markersize=10,
            markeredgecolor='black', markeredgewidth=0.5
        )
    
    # Connect with line
    ax.plot(
        results_df['temperature'], results_df['conformer_ratio'],
        '-', color='gray', linewidth=1.5, alpha=0.5, zorder=0
    )
    
    ax.set_xlabel('temperature / °C', fontsize=13, fontweight='bold')
    ax.set_ylabel(
        '$A_\\mathrm{gauche}$ / ($A_\\mathrm{gauche}$ + $A_\\mathrm{anti}$)',
        fontsize=13,
        fontweight='bold'
    )
    ax.set_ylim(0, 1)
    ax.tick_params(axis='both', which='both', direction='in',
                  top=True, right=True, labeltop=False, labelright=False)
    
    plt.tight_layout()
    plt.savefig(
        os.path.join(data_folder, 'Output_3_Conformer_Ratio.png'),
        dpi=OUTPUT_DPI,
        bbox_inches='tight'
    )
    plt.close()
    
    print("✓ Saved: Output_3_Conformer_Ratio.png")


def save_results_csv(results_df: pd.DataFrame, data_folder: str):
    """
    Save analysis results to CSV file.
    
    Args:
        results_df: Results DataFrame
        data_folder: Output folder path
    """
    print("\n" + "="*80)
    print("SAVING RESULTS")
    print("="*80)
    
    output_file = os.path.join(data_folder, 'Conformer_Analysis_Results.csv')
    results_df.to_csv(output_file, index=False, float_format='%.6f')
    
    print(f"✓ Saved: Conformer_Analysis_Results.csv")
    print(f"\nColumns:")
    for col in results_df.columns:
        print(f"  - {col}")


def print_analysis_summary(
    results_df: pd.DataFrame,
    optimal_fraction: float,
    filter_temp_min: Optional[float],
    filter_temp_max: Optional[float],
    color_temp_min: Optional[float],
    color_temp_max: Optional[float]
):
    """
    Print comprehensive analysis summary.
    
    Args:
        results_df: Results DataFrame
        optimal_fraction: Optimal Gaussian/Lorentzian fraction
        filter_temp_min: Data filtering minimum temperature
        filter_temp_max: Data filtering maximum temperature
        color_temp_min: Color scale minimum temperature
        color_temp_max: Color scale maximum temperature
    """
    print("\n" + "="*80)
    print("ANALYSIS COMPLETE!")
    print("="*80)
    print(f"\nOutput files created:")
    print(f"  1A. Output_1A_Temperature_Spectra_Normalized.png "
          f"(by conformer peak areas)")
    print(f"  1B. Output_1B_Temperature_Spectra_Raw.png")
    print(f"  2A. Output_2A_Best_Fit.png")
    print(f"  2B. Output_2B_Worst_Fit.png")
    print(f"  3.  Output_3_Conformer_Ratio.png")
    print(f"  4.  Conformer_Analysis_Results.csv")
    print(f"  5.  Outlier_Detection/ (folder with outlier detection plots)")
    
    print(f"\nConfiguration:")
    print(f"  Outlier threshold: {OUTLIER_THRESHOLD_SIGMA}σ")
    print(f"  Total outliers excluded: {results_df['n_outliers_excluded'].sum():.0f}")
    if filter_temp_min is not None and filter_temp_max is not None:
        print(f"  Data filtering: {filter_temp_min:.0f}°C to "
              f"{filter_temp_max:.0f}°C (manual)")
    else:
        print(f"  Data filtering: All data included (automatic)")
    
    if color_temp_min is not None and color_temp_max is not None:
        print(f"  Color scale: {color_temp_min:.0f}°C to "
              f"{color_temp_max:.0f}°C (manual)")
    else:
        print(f"  Color scale: {results_df['temperature'].min():.0f}°C to "
              f"{results_df['temperature'].max():.0f}°C (automatic)")
    
    print(f"\nData Summary:")
    print(f"  Spectra analyzed: {len(results_df)}")
    print(f"  Temperature range: {results_df['temperature'].min():.0f}°C - "
          f"{results_df['temperature'].max():.0f}°C")
    print(f"  Optimal fraction: {optimal_fraction:.2f} "
          f"({optimal_fraction*100:.0f}% Gauss, "
          f"{(1-optimal_fraction)*100:.0f}% Lorentz)")
    print(f"  Number of peaks per spectrum: "
          f"min={results_df['n_peaks'].min():.0f}, "
          f"max={results_df['n_peaks'].max():.0f}")
    print(f"  Average baseline: {results_df['baseline'].mean():.2f} ± "
          f"{results_df['baseline'].std():.2f}")
    
    print(f"\nConformer Analysis:")
    print(f"  Conformer ratio range:")
    min_idx = results_df['conformer_ratio'].idxmin()
    max_idx = results_df['conformer_ratio'].idxmax()
    print(f"    Minimum: {results_df['conformer_ratio'].min():.3f} at "
          f"{results_df.loc[min_idx, 'temperature']:.0f}°C")
    print(f"    Maximum: {results_df['conformer_ratio'].max():.3f} at "
          f"{results_df.loc[max_idx, 'temperature']:.0f}°C")
    
    print(f"\nFit Quality:")
    best_idx = results_df['chi_square'].idxmin()
    worst_idx = results_df['chi_square'].idxmax()
    print(f"  Best fit: {results_df.loc[best_idx, 'temperature']:.0f}°C "
          f"(χ² = {results_df['chi_square'].min():.2e})")
    print(f"  Worst fit: {results_df.loc[worst_idx, 'temperature']:.0f}°C "
          f"(χ² = {results_df['chi_square'].max():.2e})")


# =============================================================================
# MAIN WORKFLOW
# =============================================================================

def main():
    """Execute main analysis workflow."""
    # Clean up any existing figures
    plt.close('all')
    
    # Step 1: Folder and temperature selection
    data_folder = select_data_folder()
    filter_temp_min, filter_temp_max, color_temp_min, color_temp_max = \
        select_temperature_range()
    
    # Cleanup GUI resources
    print("\nCleaning up after temperature configuration...")
    cleanup_gui_resources()
    print("✓ Cleanup complete")
    
    # Print analysis configuration
    print("\n" + "="*80)
    print("RAMAN SPECTROSCOPY ANALYSIS - TWO-ROUND FITTING")
    print("="*80)
    print(f"Data folder: {data_folder}")
    print("\nCONFORMER ASSIGNMENT:")
    print("  Gauche conformer: peaks < 740 cm⁻¹")
    print("  Anti conformer:   peaks 740-790 cm⁻¹")
    print("  Dihydrogen phosphate: peaks ≥ 790 cm⁻¹ "
          "(excluded from conformer ratio)")
    print("\nDATA PROCESSING:")
    print(f"  1. Baseline correction: arPLS with linear shift")
    print("  2. Outlier detection: residual-based after first fit (threshold=3σ)")
    print("  3. Two-round fitting: refit without outliers")
    print("\nFITTING CONSTRAINTS:")
    print("  Peak position: ±5 cm⁻¹ from clicked position")
    print("  Peak width: 1-20 cm⁻¹")
    print("  Peak amplitude: 0-300% of clicked height")
    print("  Baseline: fitted constant offset")
    print("\nNORMALIZATION:")
    print("  Spectra normalized by sum of conformer peak areas "
          "(gauche + anti, excluding phosphate)")
    
    # Step 2: Load and process data
    (all_x, all_y_corrected, all_temperatures, valid_files) =         load_and_correct_spectra(data_folder)
    
    # Step 3: Filter by temperature
    (all_x, all_y_corrected, all_temperatures, valid_files) =         filter_by_temperature(
            (all_x, all_y_corrected, all_temperatures, valid_files),
            filter_temp_min,
            filter_temp_max
        )
    
    # Step 4: Interactive peak picking
    (peak_region_x, peak_region_y, peak_region_y_original, peak_region_outliers,
     all_picked_peaks, all_x, all_y_corrected, 
     all_temperatures, valid_files) = perform_interactive_peak_picking(
        all_x, all_y_corrected, all_temperatures, valid_files
    )
    
    # Step 5: Switch to non-interactive plotting
    print("\nSwitching to non-interactive plotting mode...")
    matplotlib.use('Agg')
    plt.ioff()
    
    # Step 6: Optimize Voigt fraction
    optimal_fraction = optimize_voigt_fraction(
        peak_region_x, peak_region_y, peak_region_outliers, all_picked_peaks
    )
    
    # Step 7: Fit all spectra
    results_df, all_fit_params = fit_all_spectra(
        peak_region_x, peak_region_y, peak_region_y_original, peak_region_outliers, 
        all_temperatures, valid_files, all_picked_peaks, optimal_fraction, data_folder
    )
    
    # Step 8: Filter duplicates
    (results_df, all_fit_params, peak_region_x, peak_region_y,
     peak_region_y_original) = filter_duplicate_temperatures(
        results_df, all_fit_params, peak_region_x, peak_region_y,
        peak_region_y_original
    )
    
    # Step 9: Normalize spectra
    all_y_normalized = normalize_spectra(
        peak_region_x, peak_region_y, results_df
    )
    
    # Step 10: Create plots
    temperatures = results_df['temperature'].values
    colors, temp_range_min, temp_range_max = create_temperature_colormap(
        temperatures, color_temp_min, color_temp_max
    )
    
    plot_temperature_spectra(
        peak_region_x, peak_region_y, all_y_normalized,
        temperatures, colors, data_folder, all_fit_params
    )
    
    plot_best_worst_fits(
        results_df, all_fit_params, optimal_fraction, data_folder
    )
    
    plot_conformer_ratio(
        results_df, temp_range_min, temp_range_max, data_folder
    )
    
    # Step 11: Save results
    save_results_csv(results_df, data_folder)
    
    # Step 12: Print summary
    print_analysis_summary(
        results_df, optimal_fraction,
        filter_temp_min, filter_temp_max,
        color_temp_min, color_temp_max
    )
    
    # Final cleanup
    plt.close('all')
    print("\n" + "="*80)


if __name__ == "__main__":
    main()

FOLDER SELECTION
Please select the folder containing your Raman spectroscopy data...
  Found 27 .txt file(s)

✓ Selected folder: C:/Users/janis/Documents/Dokumente/Raman_impedance/27nov25_firstheating

TEMPERATURE RANGE CONFIGURATION
Configure temperature ranges for analysis and color coding

✓ Manual mode selected:
  Data range: -40 to 140 °C
  Color scale: -40 to 140°C

Cleaning up after temperature configuration...
✓ Cleanup complete

RAMAN SPECTROSCOPY ANALYSIS - TWO-ROUND FITTING
Data folder: C:/Users/janis/Documents/Dokumente/Raman_impedance/27nov25_firstheating

CONFORMER ASSIGNMENT:
  Gauche conformer: peaks < 740 cm⁻¹
  Anti conformer:   peaks 740-790 cm⁻¹
  Dihydrogen phosphate: peaks ≥ 790 cm⁻¹ (excluded from conformer ratio)

DATA PROCESSING:
  1. Baseline correction: arPLS with linear shift
  2. Outlier detection: residual-based after first fit (threshold=3σ)
  3. Two-round fitting: refit without outliers

FITTING CONSTRAINTS:
  Peak position: ±5 cm⁻¹ from clicked position

C:\Users\janis\anaconda3\envs\raman\Lib\site-packages\rampy\baseline.py:381: RuntimeWarning: overflow encountered in exp
  wt = 1.0 / (1 + np.exp(2 * (d - (2 * s - m)) / s))
C:\Users\janis\anaconda3\envs\raman\Lib\site-packages\rampy\baseline.py:381: RuntimeWarning: overflow encountered in exp
  wt = 1.0 / (1 + np.exp(2 * (d - (2 * s - m)) / s))


  ✓ 110°C - 110C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓ 115°C - 115C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓ 120°C - 120C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt


C:\Users\janis\anaconda3\envs\raman\Lib\site-packages\rampy\baseline.py:381: RuntimeWarning: overflow encountered in exp
  wt = 1.0 / (1 + np.exp(2 * (d - (2 * s - m)) / s))
C:\Users\janis\anaconda3\envs\raman\Lib\site-packages\rampy\baseline.py:381: RuntimeWarning: overflow encountered in exp
  wt = 1.0 / (1 + np.exp(2 * (d - (2 * s - m)) / s))
C:\Users\janis\anaconda3\envs\raman\Lib\site-packages\rampy\baseline.py:381: RuntimeWarning: overflow encountered in exp
  wt = 1.0 / (1 + np.exp(2 * (d - (2 * s - m)) / s))


  ✓ 125°C - 125C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓ 130°C - 130C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓ 135°C - 135C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt


C:\Users\janis\anaconda3\envs\raman\Lib\site-packages\rampy\baseline.py:381: RuntimeWarning: overflow encountered in exp
  wt = 1.0 / (1 + np.exp(2 * (d - (2 * s - m)) / s))
C:\Users\janis\anaconda3\envs\raman\Lib\site-packages\rampy\baseline.py:381: RuntimeWarning: overflow encountered in exp
  wt = 1.0 / (1 + np.exp(2 * (d - (2 * s - m)) / s))


  ✓ 140°C - 140C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  15°C - 15C_Chol_H2PO4_linkam_imp_785_5ac_10s_50%_ULWD_50x.txt
  ✓  20°C - 20C_Chol_H2PO4_linkam_imp_785_5ac_10s_50%_ULWD_50x.txt
  ✓  25°C - 25C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  30°C - 30C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  35°C - 35C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  35°C - 35C_second_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  40°C - 40C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  45°C - 45C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  50°C - 50C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  55°C - 55C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  60°C - 60C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  65°C - 65C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  70°C - 70C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_ULWD_50x.txt
  ✓  75°C - 75C_Chol_H2PO4_linkam_imp_785_3ac_10s_50%_